KNMI. (1911–2021b, januari 1–december 31). Dagwaarden van weerstations [De Bilt]. Ministerie van Infrastructuur en Waterstaat. https://daggegevens.knmi.nl/

KNMI. (1911–2021a). Zware stormen in Nederland sinds 1910 [Dataset]. Ministerie van Infrastructuur en Waterstaat. https://www.knmi.nl/nederland-nu/klimatologie/lijsten/zwarestormen

In [1131]:
from numpy import NaN
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

Lees de daggegevens van knmi in

In [1132]:
daggegevensDf = pd.read_csv("daggegevens.csv", skiprows=46, index_col=0) # Sla de eerste 46 regels over
print(daggegevensDf)

       YYYYMMDD  DDVEC  FHVEC     FG    FHX   FHXH    FHN   FHNH    FXX  \
# STN                                                                     
260    19110101    254     46     57     72      9     31     24          
260    19110102    281     21     31     46      1     10     21          
260    19110103     51     51     57     72     17     41      4          
260    19110104     50     51     51     82     15     36     21          
260    19110105     51     31     31     51     12     10     23          
...         ...    ...    ...    ...    ...    ...    ...    ...    ...   
260    20211227    141     33     33     40     11     30      1     80   
260    20211228    216     29     45     70     13     30      1    170   
260    20211229    231     30     37     50     18     20      9    100   
260    20211230    236     64     64     80     12     50      2    150   
260    20211231    227     55     55     70      7     40     17    140   

        FXXH  ...   VVNH

C:\Users\danie\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,13,15,16,17,18,19,20,21,22,23,24,27,29,30,31,32,33,34,36,37,38,39,40) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Scrape de zware stormen

In [1133]:

url = "https://www.knmi.nl/nederland-nu/klimatologie/lijsten/zwarestormen" 
response = requests.get(url) 
html = BeautifulSoup(response.text, "html.parser") 
table = html.find_all('table')

stormenDf = pd.read_html(str(table)) # Lees de tabel in
stormenDf = pd.DataFrame(stormenDf[0])
stormenList =  []

for index, row in stormenDf.iterrows():
    stormenList.append([row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[9]]) # Voeg de tabel toe aan de stormenList

stormenDf = pd.DataFrame(stormenList) # Zet de stormenList om naar een DataFrame
stormenDf.to_csv("stormen.csv", index=False)

In [1134]:
# Maanden omzetten naar cijfers
for col in stormenDf:
    stormenDf[1] = stormenDf[1].str.replace(' jan', '01')
    stormenDf[1] = stormenDf[1].str.replace(' feb', '02')
    stormenDf[1] = stormenDf[1].str.replace(' mrt', '03')
    stormenDf[1] = stormenDf[1].str.replace(' apr', '04')
    stormenDf[1] = stormenDf[1].str.replace(' mei', '05')
    stormenDf[1] = stormenDf[1].str.replace(' jun', '06')
    stormenDf[1] = stormenDf[1].str.replace(' jul', '07')
    stormenDf[1] = stormenDf[1].str.replace(' aug', '08')
    stormenDf[1] = stormenDf[1].str.replace(' sep', '09')
    stormenDf[1] = stormenDf[1].str.replace(' okt', '10')
    stormenDf[1] = stormenDf[1].str.replace(' nov', '11')
    stormenDf[1] = stormenDf[1].str.replace(' dec', '12')

# Nul toevoegen aan ééncijferige data (bijvoorbeeld 1 oktober, 110 --> 0101)
for index, row in stormenDf.iterrows():
    if len(row[1]) < 4:
        stormenDf.iat[index, 1] = '0' + stormenDf.iloc[index][1]

print(stormenDf)
stormenDf.to_csv('stormen2.csv', index=False)

       0           1   2   3    4     5      6                 7      8
0   1911  3009/  110  11  30  108  38.0  137.0  Hoek van Holland    NaN
1   1912        2708  10  27   97  41.0  148.0  Hoek van Holland    NaN
2   1913     26/2712  10  28  101   NaN    NaN        Vlissingen    NaN
3   1914        1208  10  26   94   NaN    NaN        Den Helder    NaN
4   1914        1111  10  26   94   NaN    NaN        Vlissingen    NaN
..   ...         ...  ..  ..  ...   ...    ...               ...    ...
61  2016        2011  10  26   94  37.0  133.0          IJmuiden    NaN
62  2017        1309  10  26   94  35.0  126.0          Vlieland    NaN
63  2018        0301  10  26   94  34.0  122.0          Vlieland    NaN
64  2018        1801  11  30  108  40.0  144.0  Hoek van Holland    NaN
65  2020        0902  10  25   90  36.0  130.0          Vlieland  Ciara

[66 rows x 9 columns]


Sommige stormen duren twee dagen. Ik bewerk het csv bestand handmatig: stond er bijvoorbeeld eerst als datum "3009/   110", dan worden dat nu twee rijen met datum 30-09 en 01-10.

In [1135]:
stormenDf3 = pd.read_csv ('stormen3.csv', dtype=str)

stormenDf3 = stormenDf3.rename(columns={'0': 'YYYYMMDD',
                                        '1': 'Datum',
                                        '2': 'Hoogste windsnelheid Bft',
                                        '3': 'Hoogste uur m/s',
                                        '4': 'Hoogste uur km/h',
                                        '5': 'Zwaarste windstoot m/s',
                                        '6': 'Zwaarste windstoot km/h',
                                        '7': 'Plaats',
                                        '8': 'Naam',})

De kolom jaartal en datum worden samengevoegd zodat hij overeenkomt met dagwaarden dataframe: YYYYMMDD

In [1136]:
stormenDf3['Date'] = stormenDf3['Datum'].str[:2]
stormenDf3['Month'] = stormenDf3['Datum'].str[2:]
stormenDf3['YYYYMMDD'] = stormenDf3['YYYYMMDD'] + stormenDf3['Month'] + stormenDf3['Date']

del stormenDf3['Datum']
del stormenDf3['Month']
del stormenDf3['Date']

print(stormenDf3)

    YYYYMMDD Hoogste windsnelheid Bft Hoogste uur m/s Hoogste uur km/h  \
0   19110930                       11              30              108   
1   19111001                       11              30              108   
2   19120827                       10              27               97   
3   19131226                       10              28              101   
4   19131227                       10              28              101   
..       ...                      ...             ...              ...   
73  20161120                       10              26               94   
74  20170913                       10              26               94   
75  20180103                       10              26               94   
76  20180118                       11              30              108   
77  20200209                       10              25               90   

   Zwaarste windstoot m/s Zwaarste windstoot km/h            Plaats   Naam  
0                    38.0         

Ik maak een nieuwe kolom die bijhoudt of het heeft gestormd die dag.

In [1137]:
stormTrueOrFalse = []
for i in range(len(stormenDf3)):
    stormTrueOrFalse.append(1) # Voeg telkens een 1 toe bij de stormen

stormenDf3.insert(1, 'Storm', stormTrueOrFalse)
stormenDf3['Storm'] = stormenDf3['Storm'].astype('bool') # Sla de value op als boolean

Datasets samenvoegen

In [1138]:
# del stormenDf3['Hoogste windsnelheid Bft']
# del stormenDf3['Hoogste uur m/s']
# del stormenDf3['Hoogste uur km/h']
# del stormenDf3['Zwaarste windstoot m/s']
# del stormenDf3['Zwaarste windstoot km/h']
# del stormenDf3['Plaats']
# del stormenDf3['Naam']

daggegevensDf['YYYYMMDD'] = daggegevensDf['YYYYMMDD'].astype(str)

df = stormenDf3.merge(daggegevensDf, how='right', left_on='YYYYMMDD', right_on='YYYYMMDD')
df['Storm'] =df['Storm'].fillna(False) # Verander NaN in False
print(df)
df.to_csv('daggegevens_en_stormen.csv', index=False)

       YYYYMMDD  Storm Hoogste windsnelheid Bft Hoogste uur m/s  \
0      19110101  False                      NaN             NaN   
1      19110102  False                      NaN             NaN   
2      19110103  False                      NaN             NaN   
3      19110104  False                      NaN             NaN   
4      19110105  False                      NaN             NaN   
...         ...    ...                      ...             ...   
40538  20211227  False                      NaN             NaN   
40539  20211228  False                      NaN             NaN   
40540  20211229  False                      NaN             NaN   
40541  20211230  False                      NaN             NaN   
40542  20211231  False                      NaN             NaN   

      Hoogste uur km/h Zwaarste windstoot m/s Zwaarste windstoot km/h Plaats  \
0                  NaN                    NaN                     NaN    NaN   
1                  NaN             